# Tensorflow的保存

In [28]:
!pip install -q h5py pyyaml

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-beta0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
test_images.shape

(1000, 784)

In [4]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 只保留最终的weight
你只需要只保留最后一步的weight即可，而不需要每几步保留一下啊

In [14]:
checkpoint_path = "training_1/cp.ckpt"
                            #取名：training_1/后面是格式，相对地址，表示在当前py文件下的文件夹建立一个名叫training的文件夹
checkpoint_dir = os.path.dirname(checkpoint_path)#获取当前文件夹的绝对路径

# Create checkpoint callback object
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,         #路径
                                                 save_weights_only=True,  #只保留weight
                                                 verbose=1)
#建立一个object，class是：keras.callbacks.ModeCheckpoint
    

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training
#在fit里面也要选择保存callbacks=保存路径


#！！！！！！！！！！！！！！！！！！

#用callback是的流程
#1. 建立文件夹（path）
#2. 起名字
#3.建立一个callback的obj，class=keras.callbacks.ModelCheckpoint()
#4..fit(callbacks=[obj])



W0821 23:25:18.084122 4551808448 deprecation.py:323] From /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 704/1000 [====================>.........] - ETA: 0s - loss: 1.3439 - accuracy: 0.6023
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 598us/sample - loss: 1.1391 - accuracy: 0.6580 - val_loss: 0.7576 - val_accuracy: 0.7670
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4078 - accuracy: 0.8838
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 252us/sample - loss: 0.4169 - accuracy: 0.8870 - val_loss: 0.5113 - val_accuracy: 0.8430
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2694 - accuracy: 0.9323
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 258us/sample - loss: 0.2684 - accuracy: 0.9310 - val_loss: 0.5214 - val_accuracy: 0.8350
Epoch 4/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2068 - accuracy: 0.9495
Epoch

In [37]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002 cp.ckpt.index


In [38]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 89us/sample - loss: 2.3787 - acc: 0.1330
Untrained model, accuracy: 13.30%


In [39]:
model.load_weights(checkpoint_path)
#！！！！！只要一句话就可以回调
## model.load_weitghts（checkpoint 的路径）就瞬间直接回到该点上了


loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))



1000/1000 [==============================] - 0s 61us/sample - loss: 0.4224 - acc: 0.8610
Restored model, accuracy: 86.10%


## 设置回调点 
这里，我们没走几步，就保留一下当前的weight，

In [15]:
checkpoint_path="training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir=os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,perid=5)
#每5个iteration，建立一个独立的回调点



#！！！！！！！！
#默认period是没有的，因此默认只保留最后的weight。
#当period存在的时候，会周期性的保存。




In [50]:
model = create_model()
model.fit(train_images,train_labels,epochs=50,
         validation_data=(test_images,test_labels),
         verbose=0,callbacks=[cp_callback])


Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00006: saving model to training_2/cp-0006.ckpt

Epoch 00007: saving model to training_2/cp-0007.ckpt

Epoch 00008: saving model to training_2/cp-0008.ckpt

Epoch 00009: saving model to training_2/cp-0009.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00011: saving model to training_2/cp-0011.ckpt

Epoch 00012: saving model to training_2/cp-0012.ckpt

Epoch 00013: saving model to training_2/cp-0013.ckpt

Epoch 00014: saving model to training_2/cp-0014.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00019: saving model t

In [51]:
! ls {checkpoint_dir}

checkpoint                       cp-0026.ckpt.data-00000-of-00002
cp-0001.ckpt.data-00000-of-00002 cp-0026.ckpt.data-00001-of-00002
cp-0001.ckpt.data-00001-of-00002 cp-0026.ckpt.index
cp-0001.ckpt.index               cp-0027.ckpt.data-00000-of-00002
cp-0002.ckpt.data-00000-of-00002 cp-0027.ckpt.data-00001-of-00002
cp-0002.ckpt.data-00001-of-00002 cp-0027.ckpt.index
cp-0002.ckpt.index               cp-0028.ckpt.data-00000-of-00002
cp-0003.ckpt.data-00000-of-00002 cp-0028.ckpt.data-00001-of-00002
cp-0003.ckpt.data-00001-of-00002 cp-0028.ckpt.index
cp-0003.ckpt.index               cp-0029.ckpt.data-00000-of-00002
cp-0004.ckpt.data-00000-of-00002 cp-0029.ckpt.data-00001-of-00002
cp-0004.ckpt.data-00001-of-00002 cp-0029.ckpt.index
cp-0004.ckpt.index               cp-0030.ckpt.data-00000-of-00002
cp-0005.ckpt.data-00000-of-00002 cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00001-of-00002 cp-0030.ckpt.index
cp-0005.ckpt.index               cp-0031.ckpt.data-00000-of-00002
cp-0006.ckpt.d

In [52]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
    #从文件目录中拿到最后一个check点
latest

'training_2/cp-0050.ckpt'

In [53]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 121us/sample - loss: 0.4907 - acc: 0.8790
Restored model, accuracy: 87.90%


### 手动保存代码

In [58]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')
    #文件目录 ./表示当前目录后面每多一个/，多个子目录,最后一个是文件名

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 123us/sample - loss: 0.4907 - acc: 0.8790
Restored model, accuracy: 87.90%


### 保存整个模型

In [59]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')
#就用这一句话就可以save全部的模型，但是拓展名一定是 .h5的格式

Epoch 1/5
1000/1000 [==============================] - 0s 259us/sample - loss: 1.1622 - acc: 0.6710
Epoch 2/5
1000/1000 [==============================] - 0s 130us/sample - loss: 0.4223 - acc: 0.8890
Epoch 3/5
1000/1000 [==============================] - 0s 217us/sample - loss: 0.2959 - acc: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 173us/sample - loss: 0.2055 - acc: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 125us/sample - loss: 0.1482 - acc: 0.9720


In [60]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0813 23:02:21.777347 4525512128 deprecation.py:506] From /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0813 23:02:21.779468 4525512128 deprecation.py:506] From /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [61]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 134us/sample - loss: 0.4479 - acc: 0.8550
Restored model, accuracy: 85.50%




此技巧可保存以下所有内容：

权重值

模型配置（架构）

优化器配置

Keras 通过检查架构来保存模型。目前，它无法保存 TensorFlow 优化器（来自tf.train）。

使用此类优化器时，您需要在加载模型后对其进行重新编译，使优化器的状态变松散